In [34]:
import pandas as pd

In [35]:
data = pd.read_parquet("cleaned_data.parquet")
data.head()

,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,DEST_AIRPORT_ID,DEST_CITY_NAME,DEST_STATE_ABR,CRS_DEP_TIME,...,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,DIV_AIRPORT_LANDINGS,DIV_REACHED_DEST,DIV_ARR_DELAY,DIV_DISTANCE,DIV1_AIRPORT,DIV2_AIRPORT
0,2025-09-01,AA,1188,10721,"Boston, Ma",MA,12892,"Los Angeles, Ca",CA,1205,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,None,None
1,2025-09-01,AA,4,12892,"Los Angeles, Ca",CA,12478,"New York, Ny",NY,1520,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,None,None
2,2025-09-01,AA,12,12892,"Los Angeles, Ca",CA,10721,"Boston, Ma",MA,835,...,0.0,7.0,0.0,0.0,0.0,NaN,NaN,NaN,None,None
3,2025-09-01,AA,1578,10721,"Boston, Ma",MA,12892,"Los Angeles, Ca",CA,1807,...,0.0,0.0,0.0,20.0,0.0,NaN,NaN,NaN,None,None
4,2025-09-01,AA,28,12892,"Los Angeles, Ca",CA,12478,"New York, Ny",NY,2252,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,None,None


In [36]:
data["YEAR"] = data["FL_DATE"].dt.year
data["QUARTER"]= data["FL_DATE"].dt.quarter
data["MONTH"] = data["FL_DATE"].dt.month
data["DAY_OF_WEEK"] = data["FL_DATE"].dt.dayofweek + 1
data["DAY_OF_MONTH"] = data["FL_DATE"].dt.day
data[["FL_DATE", "YEAR", "QUARTER", "MONTH", "DAY_OF_WEEK", "DAY_OF_MONTH"]].head()

,FL_DATE,YEAR,QUARTER,MONTH,DAY_OF_WEEK,DAY_OF_MONTH
0,2025-09-01,2025,3,9,1,1
1,2025-09-01,2025,3,9,1,1
2,2025-09-01,2025,3,9,1,1
3,2025-09-01,2025,3,9,1,1
4,2025-09-01,2025,3,9,1,1


In [37]:
data["SCH_DEP_HOUR"] = (data["CRS_DEP_TIME"] // 100).astype("Int64")
data["SCH_ARR_HOUR"] = (data["CRS_ARR_TIME"] // 100).astype("Int64")

data["DEP_HOUR"] = (data["DEP_TIME"] // 100).astype("Int64")
data["ARR_HOUR"] = (data["ARR_TIME"] // 100).astype("Int64")

data[["CRS_DEP_TIME", "SCH_DEP_HOUR"]].head()

,CRS_DEP_TIME,SCH_DEP_HOUR
0,1205,12
1,1520,15
2,835,8
3,1807,18
4,2252,22


In [38]:
print(data["SCH_DEP_HOUR"].min(), data["SCH_DEP_HOUR"].max())
print(data["SCH_ARR_HOUR"].min(), data["SCH_ARR_HOUR"].max())  
print(data["DEP_HOUR"].min(), data["DEP_HOUR"].max())
print(data["ARR_HOUR"].min(), data["ARR_HOUR"].max())


0 23
0 23
0 24
0 24


In [39]:
print(data.loc[data["DEP_HOUR"] == 24])
print(data.loc[data["ARR_HOUR"] == 24])

          FL_DATE OP_UNIQUE_CARRIER  OP_CARRIER_FL_NUM  ORIGIN_AIRPORT_ID  \
1005   2025-09-01                AA               2666              11298   
1065   2025-09-01                AA               2878              11292   
2829   2025-09-01                AA               1455              11298   
7240   2025-09-01                F9               1292              11292   
23674  2025-09-02                B6               1318              12478   
55374  2025-09-04                AA               2490              12892   
57765  2025-09-04                AS                192              10299   
75884  2025-09-05                AA               2757              12892   
82407  2025-09-05                MQ               3403              11298   
109455 2025-09-07                AA               2852              14869   
109864 2025-09-07                AA               2777              12892   
116616 2025-09-07                F9               1808              13204   

In [48]:
data.loc[data["CRS_DEP_TIME"] == 2400, "CRS_DEP_TIME"] = 0
data.loc[data["CRS_ARR_TIME"] == 2400, "CRS_ARR_TIME"] = 0   
data.loc[data["DEP_TIME"] == 2400, "DEP_TIME"] = 0
data.loc[data["ARR_TIME"] == 2400, "ARR_TIME"] = 0

In [49]:
data["SCH_DEP_HOUR"] = (data["CRS_DEP_TIME"] // 100).astype("Int64")
data["SCH_ARR_HOUR"] = (data["CRS_ARR_TIME"] // 100).astype("Int64")

data["DEP_HOUR"] = (data["DEP_TIME"] // 100).astype("Int64")
data["ARR_HOUR"] = (data["ARR_TIME"] // 100).astype("Int64")

In [50]:
print(data["SCH_DEP_HOUR"].min(), data["SCH_DEP_HOUR"].max())
print(data["SCH_ARR_HOUR"].min(), data["SCH_ARR_HOUR"].max())  
print(data["DEP_HOUR"].min(), data["DEP_HOUR"].max())
print(data["ARR_HOUR"].min(), data["ARR_HOUR"].max())


0 23
0 23
0 23
0 23


In [51]:
def time_bucket(hour):
    if pd.isna(hour):
        return pd.NA
    elif 0 >= hour <= 5:
        return "Late Night/Early Morning"
    elif 6 <= hour <= 11:
        return "Morning"
    elif 12 <= hour <= 16:
        return "Afternoon"
    elif 17 <= hour <= 20:
        return "Evening"
    else:
        return "Night"
        
data["TIME_BUCKET"] = data["SCH_DEP_HOUR"].apply(time_bucket)
data["TIME_BUCKET"].value_counts()

TIME_BUCKET
Morning                     225874
Afternoon                   168312
Evening                     121426
Night                        46242
Late Night/Early Morning       585
Name: count, dtype: int64

In [ ]:
# Some cancelled flights have delay data. This will cause issues when we try to analysis delay.
cancelled_mask = data["CANCELLED"] == 1

delay_cols = [
    "DEP_DELAY_NEW",
    "ARR_DELAY_NEW",
    "DEP_DEL15",
    "ARR_DEL15",
]

data.loc[cancelled_mask, delay_cols] = pd.NA

data.groupby("CANCELLED")["DEP_DEL15"].unique()

CANCELLED
0.0    [0.0, 1.0]
1.0         [nan]
Name: DEP_DEL15, dtype: object

In [71]:
print(data.loc[data["DEP_DEL15"] == 1, "DEP_DELAY_NEW"].min())
print(data.loc[data["DEP_DEL15"] == 0, "DEP_DELAY_NEW"].max())
print(data.loc[data["ARR_DEL15"] == 1, "ARR_DELAY_NEW"].min())
print(data.loc[data["ARR_DEL15"] == 0, "ARR_DELAY_NEW"].max())

15.0
14.0
15.0
14.0


In [ ]:
data.loc[(data["DEP_DEL15"] == 1) & (data["DEP_DELAY_NEW"] < 15), "DEP_DEL15"] = 0
data.loc[(data["DEP_DEL15"] == 0) & (data["DEP_DELAY_NEW"] > 15), "DEP_DEL15"] = 1
data.loc[(data["ARR_DEL15"] == 1) & (data["ARR_DELAY_NEW"] < 15), "ARR_DEL15"] = 0
data.loc[(data["ARR_DEL15"] == 0) & (data["ARR_DELAY_NEW"] > 15), "ARR_DEL15"] = 1

data.groupby("CANCELLED")["DEP_DEL15"].unique()

